<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js083_adatomKMC2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js083_adatomKMC2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.11.25 created,  last updated on 2024.11.02
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js083_adatomKMC2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js083_adatomKMC2D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.01.15 created, last updated on 2018.11.30
//    ver 0.0.1  2019.01.23 v1, last updated on 2021.08.12
//    ver 0.0.2  2021.11.05 v2, last updated on 2021.11.05
//    ver 0.0.3  2023.04.26 v3, last updated on 2023.09.04
//
// -------------------- kinetic Monte-Carlo algorithm KMC2D
//
//  Bortz-Kalos-Liebowitz (BKL) algorithm
//
//    i-th transition rate in the system : ri
//    total transition rate : RN = sum(ri, {i=1,2,...,N})
//
//  procedure
//
//    (1) t = 0
//
//    (2) select i : randomly (proportional to ri/RN)
//      calculate R(i) = sum(rj, {j=1,2,...,i})
//      get a uniform random number u in [0,1]
//      find i  R(i-1) < u RN <= R(i)
//
//    (3) execute i-th transition
//
//    (4) update the time
//      get a uniform random numner u in (0,1]
//      dt = -log(u)/R
//      t = t + dt
//
//    goto (2)
//
//    transion rate k = nue0*exp(-dE/kT)
//      nue0 = kT/h , h: Plank's constant
//
//    periocic condition
//
// --------------------
*/

const adatomKMC2D = (function(){ // ====================  adatomKMC2D Module  ====================

	const g_hPlank = 6.62607015e-34;			// (Js) Plank's constant
	const g_EE = 1.602176634e-19;				// (C) electron charge, energy : 1(eV) = EE(J)
	const g_nMax = 10000;						// array max
	const g_nxMax = 200;						// NNx max
	const g_nyMax = 200;						// NNy max

	let g_NNa = 600;							// number of adatom
	let g_NNx = 40;								// x-division of the field
	let g_NNy = 40;								// y-division of the field
	let g_sysTime = 0.0;						// (s) system time
	let g_qqHopping = 0.2;						// (eV) hopping energy
	let g_qqNeighbor = 0.3;						// (eV) adsorption energy to neighbor adatom
	let g_kT = 0.1;								// (eV) thermal energy to random walk

	const g_lattice = dimInt2( g_nxMax, g_nyMax ); // lattice[ix][iy]  0...NNa: adatom, -1:free space
	const g_ixAdatom = dimInt1( g_nMax) ;		// g_ixAdatom[i] x-position of i-th adatom
	const g_iyAdatom = dimInt1( g_nMax );		// g_iyAdatom[i] y-position of i-th adatom
	const g_transition = dim1( 4*g_nMax );		// transition rate of direction E/N/W/S adatom
	const g_rate = dim1( 5 );					// g_rate[] = transition rate ( see setRate() )

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		if (theme==0) {
			g_NNx = 40; g_NNy = 40;
		} else if (theme==1) {
			g_NNx = 60; g_NNy = 60;
		} else if (theme==2) {
			g_NNx = 80; g_NNy = 80;
		}
		g_NNa = Math.floor(g_NNx*g_NNy*0.37);
		g_sysTime = 0.0;
		clearField();
		setParticles();
	}

	function clearField() {
		const nnx=g_NNx, nny=g_NNy;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_lattice[i][j] = -1;
			}
		}
	}

	function setParticles() {
		const nna=g_NNa, nnx=g_NNx, nny=g_NNy;
		for (let ia=0; ia<nna; ia++) {
			let x, y;
			do {
				x = Math.floor(nnx*Math.random());
				y = Math.floor(nny*Math.random());
			} while (g_lattice[x][y]!=-1); // lattice[][] >=0: particle, -1:free space
			g_lattice[x][y] = ia;
			g_ixAdatom[ia] = x;
			g_iyAdatom[ia] = y;
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc, qqHop, qqNbr, TEnergy ) {
		g_qqHopping = qqHop;
		g_qqNeighbor = qqNbr;
		g_kT = TEnergy;
		const nn = g_NNa/10 + 1;
		for (let i=0; i<nCalc; i++) {
			for (let ii=0; ii<nn; ii++) {
				const deltat = kmcStep();
				g_sysTime += deltat;
			}
		}
	}

	function kmcStep() {
		const rsum = setAllTransition();
		const ipp = selectTransion(rsum);
		move(ipp);
		const r = 1.0-Math.random(); // r=(0,1], Math.random()= [0,1)
		return(-Math.log(r)/rsum);
	}

	function setAllTransition() {
		const nna=g_NNa;

		setRate();
		let rsum = 0.0;
		for (let ia=0; ia<nna; ia++) {
			rsum += setTransition(ia);
		}
		return rsum;
	}

	function setRate() {
		g_rate[0] = transitionRate(g_qqHopping);
		g_rate[1] = transitionRate(g_qqHopping + g_qqNeighbor);
		g_rate[2] = transitionRate(g_qqHopping + 2.0*g_qqNeighbor);
		g_rate[3] = transitionRate(g_qqHopping + 3.0*g_qqNeighbor);
		g_rate[4] = transitionRate(g_qqHopping + 4.0*g_qqNeighbor);
	}

	function transitionRate(energy) {
		const nue0 = g_kT*g_EE/g_hPlank;
		return nue0*Math.exp(-energy/g_kT);
	}

	function setTransition(ia) {
		const i = g_ixAdatom[ia], j = g_iyAdatom[ia];
		const nb = numberOfNeighbor(i,j);
		g_transition[ia*4  ] = dirRate((i+1)%g_NNx, j, nb);
		g_transition[ia*4+1] = dirRate((i-1+g_NNx)%g_NNx, j, nb);
		g_transition[ia*4+2] = dirRate(i, (j+1)%g_NNy, nb);
		g_transition[ia*4+3] = dirRate(i, (j-1+g_NNy)%g_NNy, nb);
		return (g_transition[ia*4+0]+g_transition[ia*4+1]+g_transition[ia*4+2]+g_transition[ia*4+3]);
	}

	function dirRate(ii, jj, nb) {
		let nb1, rt=0.0;
		if (g_lattice[ii][jj]>=0) {
			rt = 0.0;
		} else {
			nb1 = numberOfNeighbor(ii,jj)-1;
			rt = (nb<=nb1) ? g_rate[0] : g_rate[nb-nb1];
		}
		return rt;
	}

	function numberOfNeighbor(i,j) {
		let nb=0;
		if (g_lattice[(i+1)%g_NNx][j]>=0) nb += 1;
		if (g_lattice[(i-1+g_NNx)%g_NNx][j]>=0) nb += 1;
		if (g_lattice[i][(j+1)%g_NNy]>=0) nb += 1;
		if (g_lattice[i][(j-1+g_NNy)%g_NNy]>=0) nb += 1;
		return nb;
	}

	function selectTransion(rsum) { /* find i  R(i-1) < u RN <= R(i) */
		const ipmax=4*g_NNa;
		let rnd = rsum*Math.random();
		for (let ipp=0; ipp<ipmax; ipp++) {
			rnd -= g_transition[ipp];
			if (rnd<0.0) return(ipp);
		}
		return (ipmax-1);
	}

	function move(ipp) { // update g_ixAdatom[], g_iyAdatom[], lattice[][]

		const ia = Math.floor(ipp/4);
		const id = ipp%4;
		let i = g_ixAdatom[ia], j = g_iyAdatom[ia];
		g_lattice[i][j] = -1;
		if (id==0) i = (i+1)%g_NNx;
		if (id==1) i = (i-1+g_NNx)%g_NNx;
		if (id==2) j = (j+1)%g_NNy;
		if (id==3) j = (j-1+g_NNy)%g_NNy;
		g_ixAdatom[ia] = i; g_iyAdatom[ia] = j;
		g_lattice[i][j] = ia;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( nCalc, qqHop, qqNbr, TEnergy )

		getSysParam:	function() { return [ g_NNa, g_NNx, g_NNy ]; },
		getNow:			function() { return [ g_sysTime, g_qqHopping, g_qqNeighbor, g_kT ]; },
		getWalker:		function(i) { return [ g_ixAdatom[i], g_iyAdatom[i] ]; },
		getLattice:		function(ix,iy) { return g_lattice[ix][iy]; },
	};

})(); // ====================  adatomKMC2D end  ====================


const js083 = (function(){ // ====================  js Module  ====================

	const theModule = adatomKMC2D;
	const xCanvasSize = 500;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_theme = 0;			// lattice 0:40x40, 1:60x60, 2:80x80
	let v_nCalc = 1;			// 1 (calc/frame)
	let v_qqHop = 0.20;			// (eV) hopping energy
	let v_qqNbr = 0.30;			// (eV) adsorption energy to neighbor adatom
	let v_TEnergy = 0.100;		// (eV) thermal energy

	let p_NNa, p_NNx, p_NNy; 	// <-- theModule.getSysParam()

  let sysTime, qqHopping, qqNeighbor, kT;
	let nowData = [];
  let ixList = [];
  let iyList = [];

	let dispMode = 0; // no use
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	//let inStepFlag = false;

  let breakFlag = false;
  let perticleFlag = true;

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d', { willReadFrequently:true } );
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme );
			[ p_NNa, p_NNx, p_NNy ] = theModule.getSysParam();
      perticleFlag = true;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc, v_qqHop, v_qqNbr, v_TEnergy );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc, v_qqHop, v_qqNbr, v_TEnergy  );
			//inStepFlag = true;
		}

		draw( ctx, dispMode );

    if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
		ixList = [];
		iyList = [];
		nowData = [ sysTime, qqHopping, qqNeighbor, kT ];
    for (let i=0; i<p_NNa; i++) {
			let ix, iy;
      [ ix, iy ] = theModule.getWalker(i);
			ixList[i] = ix;
			iyList[i] = iy;
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 60, yp = 40, xBoxSize = 360, yBoxSize = Math.floor(xBoxSize/p_NNx*p_NNy), sc = xBoxSize/p_NNx;
		[ sysTime, qqHopping, qqNeighbor, kT ] = theModule.getNow()

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// box
		ctx.strokeStyle = "#888888";
		ctx.strokeRect( xp-1, yp-1, xBoxSize+2, yBoxSize+2 );

		drawLattice( ctx, p_NNx, p_NNy, xp, yp, sc );

		ctx.fillStyle = "#888888";
		ctx.fillText(`periodic field = ${p_NNx} x ${p_NNy},`, 40, yCanvasSize-50);
		ctx.fillText(`number of adatom N = ${p_NNa}`, 250, yCanvasSize-50);
		ctx.fillText(`temp = ${kT} (eV) ~ ${(11602.8*kT).toFixed(1)} (K)`, 40, yCanvasSize-30);
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, 40, yCanvasSize-10);
	}

	function drawLattice( ctx, nnx, nny, xp, yp, sc ) {
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const fij = theModule.getLattice(i,j);
				if (fij>=0) {
					drawDisc(ctx, xp+(i+0.5)*sc,yp+((nny-j-1)+0.5)*sc,0.4*sc,"#00ff00");
				}
			}
		}
	}

	function drawDisc( ctx, x, y, r, color ) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setqqHop() {
		v_qqHop = Number(document.getElementById("range_qqHop").value);
		document.getElementById("text_qqHop").innerHTML = " " + v_qqHop.toFixed(2);
	}

	function setqqNbr() {
		v_qqNbr = Number(document.getElementById("range_qqNbr").value);
		document.getElementById("text_qqNbr").innerHTML = " " + v_qqNbr.toFixed(2);
	}

	function setTEnergy() {
		v_TEnergy = Number(document.getElementById("range_TEnergy").value);
		document.getElementById("text_TEnergy").innerHTML = " " + v_TEnergy.toFixed(3);
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setSpeed() {
		v_nCalc = 1 + document.getElementById("slct_speed").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, qqHopping, qqNeighbor, kT ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, ixList, iyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setqqHop:		setqqHop,		// setqqHop()
		setqqNbr:		setqqNbr,		// setqqNbr()
		setTEnergy:		setTEnergy,		// setTEnergy()
		setDispMode:	setDispMode,	// setDispMode()
		setSpeed:		setSpeed,		// setSpeed()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, qqHopping, qqNeighbor, kT ]
    pygetParticlesList: pygetParticlesList, //() :return [ nowData, ixList, iyList ]
	};

})(); // ====================  js083 module end  ====================


const js = js083;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js083] adatom - kinetic Monte-Carlo simulation KMC2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="500" HEIGHT="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>lattice 40x40</option><option>lattice 60x60</option>
<option>lattice 80x80</option>
</select>
    <span style="margin-right: 180px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>Q hopping =</label>
<input type="range" id="range_qqHop" min="0.05" max="0.25" value="0.2" step="0.01"
style="width:300px" list="tick_qqHop" oninput="js.setqqHop()">
<datalist id="tick_qqHop"><option value="0.05"><option value="0.1">
<option value="0.15"><option value="0.20"><option value="0.25"></datalist>
<label id="text_qqHop"> 0.20</label> (eV)
<br>

<label>Q neighbor =</label>
<input type="range" id="range_qqNbr" min="0.1" max="0.5" value="0.3" step="0.01"
style="width:300px" list="tick_qqNbr" oninput="js.setqqNbr()">
<datalist id="tick_qqNbr"><option value="0.1"><option value="0.2">
<option value="0.3"><option value="0.4"><option value="0.5"></datalist>
<label id="text_qqNbr"> 0.30</label> (eV)
<br>

<label>kT(Temp) =</label>
<input type="range" id="range_TEnergy" min="0.025" max="0.2" value="0.1" step="0.005"
style="width:300px" list="tick_TEnergy" oninput="js.setTEnergy()">
<datalist id="tick_TEnergy"><option value="0.05"><option value="0.10"><option value="0.15">
<option value="0.20"><option value="0.25"></datalist>
<label id="text_TEnergy"> 0.100</label> (eV)
<br>

<label>nCalc/frame:</label>
<select id="slct_speed" onChange="js.setSpeed()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option><option>7</option><option>8</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get system data and print
for i in range(10):
  [ sysTime, qqHopping, qqNeighbor, kT ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  time = {sysTime*1e12:>8.2f} (ps),  Q_hopping = {qqHopping:>6.2f} (eV),  Q_neighbor = {qqNeighbor:>6.2f} (eV),  Temp = {kT:>6.2f} (eV)')
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme

import time

themeList = [ '0: lattice 40x40', '1: lattice 60x60', '2: lattice 80x80' ]

# exec html-js code
exec_html_js()
print("-- start --")

# change theme
for theme in [ 0, 1, 2 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print(f'-- theme :{themeList[theme]} --')

  # get system data and print
  for i in range(3):
    [ sysTime, qqHopping, qqNeighbor, kT ] = eval_js( 'js.pygetData({})'.format(i) )
    print(f'\t  time = {sysTime*1e12:>8.2f} (ps),  Q_hopping = {qqHopping:>6.2f} (eV),  Q_neighbor = {qqNeighbor:>6.2f} (eV),  Temp = {kT:>6.2f} (eV)')
    time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get adatom data and save

import time
import numpy as np

themeList = [ '0: lattice 40x40', '1: lattice 60x60', '2: lattice 80x80' ]

# exec html-js code
exec_html_js()
print("-- start --")

# change theme
theme = 1 # '1: lattice 60x60'
eval_js( 'js.pysetTheme({})'.format(theme) )
print(f'-- theme :{themeList[theme]} --')

# get system data and print
for i in range(10):
  [ sysTime, qqHopping, qqNeighbor, kT ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  time = {sysTime*1e12:>8.2f} (ps),  Q_hopping = {qqHopping:>6.2f} (eV),  Q_neighbor = {qqNeighbor:>6.2f} (eV),  Temp = {kT:>6.2f} (eV)')
  time.sleep(1)

# get walkers data
print("")
print("-- get walkers data --")
[ nowData, ixList, iyList ] = eval_js('js.pygetParticlesList()')
[ sysTime, qqHopping, qqNeighbor, kT ] = nowData
print(f'got time = {sysTime*1e12:>8.2f} (ps),  Q_hopping = {qqHopping:>6.2f} (eV),  Q_neighbor = {qqNeighbor:>6.2f} (eV),  Temp = {kT:>6.2f} (eV)')
print(f'len(ixList) ={len(ixList)}, len(iyList) ={len(iyList)} ')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# convert [ ixList, iyList ] to np_data
np_data = np.array([ ixList, iyList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js083_data.npy'" )
np.save( 'js083_data.npy', np_data )

In [4]:
# load data and prepare numpy data for plot

import numpy as np

# load np_data <-- np.array([ ixList, iyList ])
print("-- load data --")
loaded_data = np.load('js083_data.npy')

# prepare data ( numpy array ) for plot
X = loaded_data[0]  # x-position of every adatom
Y = loaded_data[1]  # y-position of every adatom

-- load data --


In [ ]:
# scatter plot walkers / statusList, ixList, iyList

import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.scatter( X, Y )
plt.title("adatom on 60x60 crystal site")

In [6]:
# prompt: get number of neighbors list from X,Y

import numpy as np

def get_neighbors(X, Y, radius=1):
    """
    Finds the indices of neighboring points within a given radius.

    Args:
        X: A 1D numpy array of x-coordinates.
        Y: A 1D numpy array of y-coordinates.
        radius: The radius within which to search for neighbors.

    Returns:
        A list of lists, where each inner list contains the indices of the neighbors for a given point.
    """
    num_points = len(X)
    neighbors_list = [[] for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance = np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2)
                if distance <= radius:
                    neighbors_list[i].append(j)
    return neighbors_list

# Example usage (assuming you have X and Y from the previous code):
# loaded_data = np.load('js083_data.npy')
# X = loaded_data[0]
# Y = loaded_data[1]

# neighbors = get_neighbors(X, Y)
# print(neighbors)

In [ ]:
# prompt: hist number of neighbors

import matplotlib.pyplot as plt

# ... (Your existing code)

# Example usage (assuming you have X and Y from the previous code):
neighbors = get_neighbors(X, Y)

# Calculate the number of neighbors for each point
num_neighbors = [len(neighbor_indices) for neighbor_indices in neighbors]

# Create a histogram of the number of neighbors
plt.figure(figsize=(8, 6))
plt.hist(num_neighbors, bins=range(max(num_neighbors) + 2), align='left', rwidth=0.8)
plt.xlabel("Number of Neighbors")
plt.ylabel("Number of Points")
plt.title("Distribution of Number of Neighbors")
plt.show()